In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Time to get serious

![](http://cdn.vox-cdn.com/thumbor/mXo5ObKpTbHYi9YslBy6YhfedT4=/95x601:1280x1460/1200x800/filters:focal(538x858:742x1062)/cdn.vox-cdn.com/uploads/chorus_image/image/66699060/mgidarccontentnick.comc008fa9d_d.0.png)

Lets have a little fun with fastai. Im going to use a language model from fastai to generate new episodes of Avatar the last airbender. Cause we all wanted more episodes.

In [ ]:
from fastai.text.all import *

In [ ]:
df_lm = pd.read_csv('../input/avatar-the-last-air-bender/avatar.csv',encoding='latin1') 

In [ ]:
df_lm.info()

In [ ]:
df_lm.head()

In [ ]:
df_lm['chapter'].unique()

The data object rows are broken up by scene. Each full_text entry is the full text for a scene. So lets make a function that gets the full script for each episode. 

In [ ]:
def get_scripts(df):
    scripts = []
    for episode in df['chapter'].unique():
        script = ''
        for i, obj in df.iterrows():
            if obj['chapter'] == episode:
                script += obj['character'] + ": " + obj['full_text'] + " "
        
        scripts.append(script)
    return pd.DataFrame(scripts)

scripts_ = get_scripts(df_lm)
scripts_ = scripts_.apply(lambda x: x.replace("\n", " "))

In [ ]:
scripts_

In [ ]:
dls = TextDataLoaders.from_df(scripts_, is_lm=True)
dls.show_batch(max_n=3)

The TextDataLoaders object just created a dataset where the independent variable is a sequence of antecedent from the avatar corpus and the dependent variable is a sequence of words index offset by one. It also takes care of the tokenization process for us.

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, pretrained=True, drop_mult=0.5, metrics=[accuracy, perplexity])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 2e-2, 
                    #cbs=EarlyStoppingCallback(min_delta=0.005, monitor='valid_loss', patience=2)
                   )

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(8, slice(1e-5, 2e-3))

In [ ]:
learn.recorder.plot_loss()

In [ ]:
start = "The scene opens"
def write_episode(start):

    N_WORDS = 400
    N_SENTENCES = 1
    preds = [learn.predict(start, N_WORDS) 
             for _ in range(N_SENTENCES)]

    return (" ".join(preds)) + ". THE END."



In [ ]:
ep = write_episode(start)
print(ep)

THE END

![](http://imgix.bustle.com/uploads/image/2020/6/1/65220bb4-d554-403f-a682-d704bd217007-atla-iroh-tea.jpg?w=1200&h=630&fit=crop&crop=faces&fm=jpg)

More tweeks obviously need to be made. Let me know in the comments if any of you have suggestions. 